<a href="https://colab.research.google.com/github/jhbellingrath/CS-290-Classwork/blob/main/notebooks/gp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
import math

In [ ]:
df=pd.read_csv('https://raw.githubusercontent.com/jhbellingrath/CS-290-Classwork/refs/heads/main/data/survey%20lung%20cancer.csv')

In [ ]:
target="LUNG_CANCER"

In [ ]:
attributes=df.columns[df.columns!=target]
attributes=attributes.drop("AGE")
attributes

Index(['GENDER', 'SMOKING', 'YELLOW_FINGERS', 'ANXIETY', 'PEER_PRESSURE',
       'CHRONIC DISEASE', 'FATIGUE ', 'ALLERGY ', 'WHEEZING',
       'ALCOHOL CONSUMING', 'COUGHING', 'SHORTNESS OF BREATH',
       'SWALLOWING DIFFICULTY', 'CHEST PAIN'],
      dtype='object')

In [ ]:
# entropy function for qualitative
def entropy(attribute):
  entropy = 0
  total = len( df )
  vals = df[attribute].unique()
  for val in vals:
    counts = len(df[ df[attribute] == val ])
    weight = counts / total
    props = df[ df[attribute] == val ][target].value_counts( normalize=True )
  for p in props.array:
    entropy =  entropy - weight*(p*math.log2(p))
  return entropy

In [ ]:
# Gini function for qualitative
def gini(attribute):
    vals=df[attribute].unique()
    impurity=1
    for val in vals:
      impurity-=(len(df[df[attribute]==val])/len(df))**2
    return impurity

In [ ]:
#  entropy for a quantitative variable
def entropy_quantitative(attribute):
  entropies=[]
  total = len(df)
  vals = df[attribute].unique()
  for val in vals:
    entropy_value = 0
    # Left split
    left = df[df[attribute] <= val][[attribute, target]]
    props = left[target].value_counts(normalize=True)
    weight = len(left) / total
    for prop in props.array:
      entropy_value -= weight * prop * math.log2(prop)

    # Right split
    right = df[df[attribute] > val][[attribute, target]]
    props = right[target].value_counts(normalize=True)
    weight = len(right) / total
    for prop in props.array:
      entropy_value -= weight * prop * math.log2(prop)

    entropies.append([entropy_value,val])
  entropies.sort()
  return entropies[0]
  #returns a list of [entropy value,split value]

In [ ]:
# gini impurity for a quantitative variable with a pecified split value
def gini_quantitative(attribute,split_val):
  impurity = 1
  # Left split
  impurity -= (len(df[df[attribute] <= split_val]) / len(df))**2

  # Right split
  impurity -= (len(df[df[attribute] > split_val]) / len(df))**2
  return impurity

In [ ]:
def Attribute_selection_method(data,task,target,impurity_measure):
  attributes=data.columns[data.columns!=target]
  if task=="classification":
    if impurity_measure=="entropy":
      entropies=[]
      for attribute in attributes:
        if len(data[attribute].unique())<5:
          entropies.append([entropy(attribute),attribute])
        else:
          entropies.append(entropy_quantitative(attribute))
      entropies.sort()
      return "The best attribute is "+entropies[0][1]
    if impurity_measure=="gini":
      ginis=[]
      for attribute in attributes:
        if len(data[attribute].unique())<5:
          ginis.append([gini(attribute),attribute])
        else:
          best_entropy=entropy_quantitative(attribute)
          ginis.append([gini_quantitative(attribute,best_entropy[1]),attribute])
      ginis.sort(reverse=True)
      return "The best attribute is "+ginis[0][1]

In [ ]:
Attribute_selection_method(df,"classification","LUNG_CANCER","gini")

'The best attribute is PEER_PRESSURE'

In [ ]:
# LabelEncoder
le = LabelEncoder()

In [ ]:
# target variable
df["LUNG_CANCER"] = le.fit_transform(df["LUNG_CANCER"])

In [ ]:
# Compare
X = df.drop(columns=["LUNG_CANCER"])
y = df["LUNG_CANCER"]

In [ ]:
# Iterate over all columns in X
for col in X.columns:
    if X[col].dtype == 'object':  # Check if the column is object
        X[col] = le.fit_transform(X[col])  # Fit and transform


In [ ]:
clf = DecisionTreeClassifier(criterion='gini')
clf.fit(X, y)

DecisionTreeClassifier()

In [ ]:
# Get the first split attribute
first_split_attribute = X.columns[clf.tree_.feature[0]]
print("First attribute chosen by DecisionTreeClassifier:", first_split_attribute)

First attribute chosen by DecisionTreeClassifier: ALLERGY 


In [ ]:
df2=pd.read_csv('https://raw.githubusercontent.com/jhbellingrath/CS-290-Classwork/refs/heads/main/data/Housing_Prices.csv')

In [ ]:
df2.head()